V1

In [ ]:
import pandas as pd
from PyCaller import process_urls
from tqdm import tqdm

def process_and_print_results(url):
    print(f"Processing URL: {url}")
    data = process_urls([url])

    if data:
        print(f"\nResults for URL: {url}")
        for key, df in data.items():
            if df is not None and not df.empty:
                print(f"{key.replace('_', ' ').title()}:")
                print("\n")
        return data
    else:
        print(f"No data found for URL: {url}")
        return None

def main():
    # Load initial URLs
    all_found_urls_s = pd.read_csv("all_found_urls_23.12.23_cleaned.csv")
    initial_urls = list(set(all_found_urls_s["0"]))[:10]

    # Initialize DataFrames
    all_pole_studio_data = pd.DataFrame()
    all_workshops_data = pd.DataFrame()
    all_workshop_details_data = pd.DataFrame()

    # Process each URL
    total_urls = len(initial_urls)
    for i, url in tqdm(enumerate(initial_urls, start=1), total=total_urls, desc="Processing URLs"):
        print(f"Processing URL {i}/{total_urls}: {url}")
        data = process_and_print_results(url)
        if data:
            # Add data to corresponding DataFrames
            if 'pole_studio_data' in data and not data['pole_studio_data'].empty:
                all_pole_studio_data = pd.concat([all_pole_studio_data, data['pole_studio_data']], ignore_index=True)
            
            if 'workshops_data' in data and not data['workshops_data'].empty:
                all_workshops_data = pd.concat([all_workshops_data, data['workshops_data']], ignore_index=True)
            
            if 'workshop_details' in data and not data['workshop_details'].empty:
                all_workshop_details_data = pd.concat([all_workshop_details_data, data['workshop_details']], ignore_index=True)

        # Display progress
        progress_percent = (i / total_urls) * 100
        print(f"Progress: {progress_percent:.2f}%\n")

    # # Export DataFrames to CSV files
    # all_pole_studio_data.to_csv("Pole_Studio_Übersicht_S.csv", index=False)
    # all_workshops_data.to_csv("Workshop_Liste_SW.csv", index=False)
    # all_workshop_details_data.to_csv("Workshop_Übersicht_E.csv", index=False)

if __name__ == "__main__":
    main()


URLS Cleaner

In [ ]:
import pandas as pd

def clean_and_unique_urls(file_path):
    # Read the CSV file
    all_found_urls_s = pd.read_csv(file_path, header=None, names=['index', 'url'])

    # Ensure all URLs are in lowercase
    all_found_urls_s['url'] = all_found_urls_s['url'].str.lower()

    # Remove duplicate URLs
    all_found_urls_s = all_found_urls_s.drop_duplicates(subset=['url'])

    # Save the cleaned and unique URLs to a new CSV file
    cleaned_file_path = file_path.replace('.csv', '_cleaned.csv')
    all_found_urls_s.to_csv(cleaned_file_path, index=False, header=False)

    return cleaned_file_path

if __name__ == "__main__":
    # Example usage
    input_file_path = "all_found_urls_23.12.23.csv"
    cleaned_file_path = clean_and_unique_urls(input_file_path)
    print(f"Cleaned and unique URLs saved to: {cleaned_file_path}")


V2

In [ ]:
import pandas as pd
from PyCaller import process_urls
from tqdm import tqdm

def process_and_print_results(url):
    data = process_urls([url])

    if data:
        for key, df in data.items():
            if df is not None and not df.empty:
                print(f"{key.replace('_', ' ').title()}: {len(df)} entries")

def main():
    # Load initial URLs
    all_found_urls_s = pd.read_csv("all_found_urls_23.12.23_cleaned.csv")
    initial_urls = list(set(all_found_urls_s["0"]))[:100]

    # Initialize DataFrames
    all_pole_studio_data = pd.DataFrame()
    all_workshops_data = pd.DataFrame()
    all_workshop_details_data = pd.DataFrame()

    # Process each URL
    total_urls = len(initial_urls)
    with tqdm(total=total_urls, desc="Processing URLs") as pbar:
        for i, url in enumerate(initial_urls, start=1):
            process_and_print_results(url)
            pbar.update(1)

if __name__ == "__main__":
    main()


V3

In [7]:
import pandas as pd
from PyCaller import process_urls
from tqdm import tqdm

def process_and_print_results(url, all_pole_studio_data, all_workshops_data, all_workshop_details_data):
    data = process_urls([url])

    if data:
        for key, df in data.items():
            if df is not None and not df.empty:
                print(f"{key.replace('_', ' ').title()}: {len(df)} entries")

                # Update the appropriate DataFrame
                if key == 'pole_studio_data':
                    all_pole_studio_data = pd.concat([all_pole_studio_data, df], ignore_index=True)
                elif key == 'workshops_data':
                    all_workshops_data = pd.concat([all_workshops_data, df], ignore_index=True)
                elif key == 'workshop_details':
                    all_workshop_details_data = pd.concat([all_workshop_details_data, df], ignore_index=True)

    return all_pole_studio_data, all_workshops_data, all_workshop_details_data

def main():
    # Load initial URLs
    all_found_urls_s = pd.read_csv("your_output_file.csv")
    initial_urls = list(set(all_found_urls_s["0"]))

    # Initialize DataFrames
    all_pole_studio_data = pd.DataFrame()
    all_workshops_data = pd.DataFrame()
    all_workshop_details_data = pd.DataFrame()

    # Process each URL with tqdm
    with tqdm(total=len(initial_urls), desc="Processing URLs", dynamic_ncols=True) as pbar:
        for url in initial_urls:
            all_pole_studio_data, all_workshops_data, all_workshop_details_data = process_and_print_results(
                url, all_pole_studio_data, all_workshops_data, all_workshop_details_data
            )
            pbar.update(1)
            pbar.set_postfix_str(f"Current URL: {url}", refresh=True)

    # # Export DataFrames to CSV files
    all_pole_studio_data.to_csv("Pole_Studio_Übersicht_S.csv", index=False)
    all_workshops_data.to_csv("Workshop_Liste_SW.csv", index=False)
    all_workshop_details_data.to_csv("Workshop_Übersicht_E.csv", index=False)

    # Return the final DataFrames
    return all_pole_studio_data, all_workshops_data, all_workshop_details_data

# Run the main function and get the final DataFrames
result_pole_studio, result_workshops, result_workshop_details = main()

# Now you can access result_pole_studio, result_workshops, and result_workshop_details outside the function
# print(result_pole_studio)
# print(result_workshops)
# print(result_workshop_details)


Processing URLs:   0%|          | 0/403 [00:00<?, ?it/s]

Starting URL reconstruction...


2024-02-03 11:29:58,069 - INFO - Processing URL 1/1: https://www.eversports.de/s/swiss-mega-park


Error reconstructing URL for 'Sportplätze': Unrecognized text: 'Sportplätze'
Error reconstructing URL for 'Matches': Unrecognized text: 'Matches'
Converting to DataFrame...
Validating URLs...


2024-02-03 11:29:59,808 - INFO - URL https://www.eversports.de/s/swiss-mega-park is valid.


...

Error during HTTP request: 404 Client Error: Not Found for url: https://www.eversports.de/s/vertical-kay-poledance-%26-aerialarts-education
Converting to DataFrame...
Validating URLs...
Processing completed.
Starting URL reconstruction...


Processing URLs: 100%|█████████▉| 402/403 [32:50<00:04,  4.17s/it, Current URL: https://www.eversports.de/s/anni's-pole-dance-neum%c3%bcnster]              

Error during HTTP request: 404 Client Error: Not Found for url: https://www.eversports.de/s/anni's-pole-dance-neum%C3%BCnster
Converting to DataFrame...
Validating URLs...
Processing completed.
Starting URL reconstruction...


Processing URLs: 100%|██████████| 403/403 [32:51<00:00,  4.89s/it, Current URL: https://www.eversports.de/s/souldance]                        

Error during HTTP request: 404 Client Error: Not Found for url: https://www.eversports.de/s/souldance
Converting to DataFrame...
Validating URLs...
Processing completed.


In [8]:
result_pole_studio

,PoleStudio_Name,Adresse,PLZ,Stadt,Straße,Buttons,Pole Studio Beschreibung,E-Mail,Homepage,Telefon,URL_S,Art,Angebot,Created Date,Updated Date
0,SWISS MEGA PARK,"[Schützenstrasse 2, 4402 Frenkendorf]",4402,Frenkendorf,Schützenstrasse 2,"[Übersicht, Sportplätze, Matches]",Der SWISS MEGA PARK in Frenkendorf ist nicht n...,kontakt@swissmegapark.ch,http://www.swissmegapark.ch/,+41 61 901 76 64,https://www.eversports.de/s/swiss-mega-park,"[12 Rezensionen, Tennis, Badminton, Basketball...",None,2024-02-03 11:30:00,2024-02-03 11:30:00
1,POLEside,"[Saline 5 , 74177 Bad Friedrichshall]",74177,Bad,Saline 5,"[Übersicht, Klassen, Kurse, Workshops, Videos,...",Willkommen bei POLEside!Lass Dich von uns mitr...,info@poleside.de,http://poleside.de/,0176 870 191 81,https://www.eversports.de/s/poleside,"[Poledance, Fitness, Aerial Yoga, Polesport, P...",Zum Stundenplan,2024-02-03 11:30:08,2024-02-03 11:30:08
2,CSS AERIAL DANCE STUDIO,"[Hummelsbütteler Steindamm , 22851 Hamburg]",22851,Hamburg,Hummelsbütteler Steindamm,"[Übersicht, Klassen, Kurse, Preise, Team]",🌟 Herzlich willkommen im CSS AERIAL DANCE STUD...,CorinnaSchatz@gmx.de,http://www.cssaerialdancestudio.com,+4915224340258,https://www.eversports.de/s/css-aerial-dance-s...,"[Flexibles Training, Poledance, Poledance, Pol...",Zum Stundenplan,2024-02-03 11:30:24,2024-02-03 11:30:24
3,Tanzteufel,"[Karl-Friedrich-Straße 76 , 79312 Emmendingen]",79312,Emmendingen,Karl-Friedrich-Straße 76,"[Übersicht, Klassen, Videos, Preise, Team]",In den beiden großen und offenen Studios in Em...,info@tanzteufel-poledance.com,http://www.tanzteufel-poledance.com,+49 162 7817549,https://www.eversports.de/s/tanzteufel,"[Poledance, Power Pole, Poledance, Poledance, ...",1 Angebot für Neukund:innen,2024-02-03 11:30:32,2024-02-03 11:30:32
4,The Pole Jungle,"[Volksgartenstraße 19, 4020 Linz]",4020,Linz,Volksgartenstraße 19,"[Übersicht, Klassen, Trainings, Kurse, Worksho...",Wie der Name bereits sagt erwartet euch ein gr...,hello@thepolejungle.com,http://www.thepolejungle.com,+43 677 647 33 8 00,https://www.eversports.de/s/the-pole-jungle,"[Poledance, Poledance, Poledance, Poledance, P...",Zum Stundenplan,2024-02-03 11:30:39,2024-02-03 11:30:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Pole Passion Bielefeld,"[Apfelstraße 75A, 33613 Bielefeld]",33613,Bielefeld,Apfelstraße 75A,[Übersicht],"""Come in and dance"" heißt es bei Pole Passion ...",info@polepassion.de,http://www.polepassion.de/,+49 017663169504,https://www.eversports.de/s/pole-passion-biele...,"[(Noch keine Bewertungen), Poledance, Hip Hop,...",Kostenlos Schnuppern,2024-02-03 12:01:58,2024-02-03 12:01:58
156,Polea - Pole and Aerial Dance,"[Budenheimer Parkallee 5 , 55257 Budenheim]",55257,Budenheim,Budenheimer Parkallee 5,"[Übersicht, Klassen, Workshops, Videos, Preise...",Das Polea - Pole and Aerial Dance ist dein Stu...,info.polea@googlemail.com,None,+49 17645911101,https://www.eversports.de/s/polea-pole-and-aer...,"[Poledance, Aerial Yoga, Aerial Akrobatik, Tan...",Zum Stundenplan,2024-02-03 12:02:05,2024-02-03 12:02:05
157,Pole and Move Mannheim,"[Edisonstraße 20 , 68309 Mannheim]",68309,Mannheim,Edisonstraße 20,"[Übersicht, Klassen, Workshops, Preise, Team]",Willkommen bei Pole and Move. Unsere Studios b...,team@poleandmove.de,https://poleandmove.de/,+49176 61653663,https://www.eversports.de/s/pole-and-move-mann...,"[Fitness, Tanzen, Poledance, Poledance, Power ...",2 Angebote für Neukund:innen,2024-02-03 12:02:18,2024-02-03 12:02:18
158,The Supergirl Studio,"[Brunngartenstraße 5a, 85221 Dachau]",85221,Dachau,Brunngartenstraße 5a,"[Übersicht, Klassen, Preise, Team]",Trainiere mit uns im “THE SUPERGIRL STUDIO” in...,supergirl@poledance-dachau.de,http://www.poledance-dachau.de,+4915758721177,https://www.eversports.de/s/the-supergirl-studio,"[Power Pole, Polesport, Poledance, Online Klas...",2 Angebote für Neukund:innen,2024-02-03 12:02:39,2024-02-03 12:02:39


In [9]:
result_workshop_details

,Workshop Name,Description,Studio Name,Location,Level,Date,URL_E,Time
0,Workshop mit @bou_boulette (Janine Hollung),Am Samstag den 02.03.2024 dürfen wir Janine al...,POLEside,"Ticket für Sa. 02.03.2024, 16:00 - 17:30",Poledance,"Beginnt am Samstag, 2. März 2024",https://www.eversports.de/e/workshop/llCPKoMLh,"Weststraße 1, 74193 Schwaigern"
1,Einsteiger-Workshop,Nach einem gemütlichen Kennenlernen beginnen w...,Star Poledance,Einsteiger-Workshop,Poledance,"Samstag, 10. Februar 2024",https://www.eversports.de/e/workshop/jZ6LjEO,18:00 - 20:00
2,Seductive Sunday - Floorwork,"Teasen, Fühlen , stark und weiblich sein . Anm...",Da Becca Flow and Fit,Einzelticket,Floorwork,"Sonntag, 25. Februar 2024",https://www.eversports.de/e/workshop/2z_srY0,15:00 - 16:30
3,Seductive Sunday - Pole,"Teasen, Fühlen , stark und weiblich sein . Anm...",Da Becca Flow and Fit,Einzelticket,Poledance,"Sonntag, 25. Februar 2024",https://www.eversports.de/e/workshop/o5_MV7J,17:00 - 18:30
4,Rock the Pole. A Trilogie.,Ihr zahlt einmalig für alle drei stunden zusam...,Da Becca Flow and Fit,5er Karte (Neukunden bitte vorher eine email a...,Poledance,"Beginnt am Samstag, 9. März 2024",https://www.eversports.de/e/workshop/0u01Mza,"Eickeler Markt 3A, 44651 Herne"
...,...,...,...,...,...,...,...,...
302,Handstand Workshop,Du möchtest gerne den Handstand lernen und dir...,Pole and Move Mannheim,"Edisonstraße 20, 68309 Mannheim",Poledance,"Samstag, 9. März 2024",https://www.eversports.de/e/workshop/gbofUBp,12:30 - 13:40
303,Aerial Hoop Choreo Workshop,"Aerial Hoop, auch Luftring oder Lyra genannt, ...",Pole and Move Mannheim,"Edisonstraße 20, 68309 Mannheim",Aerial Hoop,"Samstag, 9. März 2024",https://www.eversports.de/e/workshop/NFfn_y6,13:45 - 14:55
304,We love Pancakes - mit Caroline Lange,Dieser Workshop beginnt mit intensiven Stretch...,Pole and Move Mannheim,"Edisonstraße 20, 68309 Mannheim",Poledance,"Freitag, 22. März 2024",https://www.eversports.de/e/workshop/miY7n0b,19:10 - 20:40
305,Aerial Hoop Choreo Workshop,"Aerial Hoop, auch Luftring oder Lyra genannt, ...",Pole and Move Mannheim,"Edisonstraße 20, 68309 Mannheim",Aerial Hoop,"Samstag, 13. April 2024",https://www.eversports.de/e/workshop/C9b62JT,12:30 - 13:40


Tools

In [6]:
import pandas as pd

def clean_and_unique_urls(input_csv, output_csv):
    # Read the CSV file
    df = pd.read_csv(input_csv, sep=',', header=None, names=["Index", "URL"])

    # Clean the URLs
    df['Cleaned_URL'] = df['URL'].apply(lambda url: '-'.join(filter(None, url.split('-'))))

    # Remove duplicate URLs
    df = df.drop_duplicates(subset=['Cleaned_URL'])

    # Save the cleaned and unique URLs to a new CSV file
    df[['Index', 'Cleaned_URL']].to_csv(output_csv, index=False, header=None)

# Example usage
input_csv = "all_found_urls_23.12.23_cleaned_cleaned.csv"  # Replace with the actual path to your CSV file
output_csv = "your_output_file.csv"  # Replace with the desired output path

clean_and_unique_urls(input_csv, output_csv)
